## In-Class Lecture 39-1 - Debugging and GDB

## Create folder

<b>To review:</b>

<b>1.</b> Open Docker Desktop. Go to the cse20133-user Container you created. Press the Blue Triangle to start the Container.

<b>2.</b> Open VSCode, and on the left, select the blue button in the bottom left of VSCode. At the top, select the pull-down choice “Attach to running container” and select your CSE 20133 course container.

<b>3.</b> Go into your Git Folder:

> Recall that @USERNAME is the unique username you created when you created your GitHub account. You will see your user name in the VS Code Docker

    cd cse20133-user/cse20133-@USERNAME

Create the folder:

    mkdir lec39
    cd lec39

### Obtaining the Files

Perform the following command:

    wget https://raw.githubusercontent.com/mmorri22/cse20133/main/inclass/lec39/setup.sh
    chmod a+rx setup.sh
    ./setup.sh
    
Once you perform these commands, you will get the following:

    gbd-part1.c gdb-part2.c

### How I Got Better at Debugging

From <a href = "https://drawings.jvns.ca/">Julia's Drawings</a>:

![](https://drawings.jvns.ca/drawings/better-debugging.png)

### Testing, Debugging, and GDB Overview

Testing is absolutely essential to a typical development process. Suppose you have just finished homework and you think you have a pretty good implementation of a doubly linked list. How do you know, for sure, that this list behaves like how you intended it to be? You can manually read through the code a million times, but you might still miss something.

Debugging involves <b>backwards reasoning</b>, like solving murder mysteries.  Something impossible occurred, and the only solid information is that it really did occur.  So we must think backwards from the result to <b>discover the reasons</b>.  Once we have a full explanation, <b>we'll know what to fix</b> and, along the way, likely discover a few other things we hadn't expected.

<b><a href = "https://www.sourceware.org/gdb/">GDB</a></b> is a program that lets you interactively trace, probe, and examine a process. 
<ul>
    <li>Step through code one line at a time</li>
    <li>Print variables</li>
    <li>Set breakpoints and watch points</li>
</ul>

   

To use gdb, compile your program with debugging symbols and then execute it using gdb:


    > gdb ./program
    ...
    (gdb) run
    (gdb) bt
    (gdb) l

### Solving a Segmentation Fault using GDB - <font color = "red">Participation Points Opportunities</fton>

Working with a partner or group of 3, you will be given a program that compiles but does not run successfully. When you try to run it, you should get the message “Segmentation fault”. This and “Bus Error” are the two most common run-time errors. Note that no indication is given as to where the program failed. You can imagine that in a large program, the message is pretty useless in trying to locate the problem.

Type the following command:

    vim gdb-part1.cpp

You will see the following code segment. Observe the condition in the for loop.

    #include <stdio.h>

    int main(void)
    {

        const int data[5] = {1, 2, 3, 4, 5};

        int i, sum;

        for (i = 0; i >= 0; ++i) {
             sum += data[i];

            fprintf( stdout, "Current sum: %d and current data: %d\n", sum, data[i] );
        }

        fprintf( stdout, "sum = %d \n", sum);

        return 0;
    }

Close out of vim using q!

Compile with only -g (meaning no PQC flags)        

    gcc -g gdb-part1.cpp -o gdb-part1

Run the program the regular way without gdb:   

    ./gdb-part1

You will see the program produces a Segmentation Fault after printing some bizarre results. These are the result of integer overflow.

Start the program with gdb:  

    gdb –tui ./gdb-part1  

Type the command:

    list main

    break main

    run 

(this starts running the program up to the break point at main)

    p data 

(the debugger stops at the line preceding the current line of code shown, so when you try to print data, at that point, the array called data has not been initialized yet; in other words, the line of code that is highlighted is the line that will be executed next).

    next

    p data   

(now the values for data are there)

    next

    next

    next

    p i

    next

    p i

    next

    p i 

(see what’s happening?)

    continue   

( the program continues until it gets to the next break, or, as in this case, the seg fault since no other break was set )

   
This is what GDB will show (with the main memory address being different based on your specific run)

    Program received signal SIGSEGV, Segmentation fault.
    0x000000000040052d in main () at example.c:11
    and the line of code that is highlighted in the upper box is:
    >│11 sum += data[i];

list this will list 10 lines near where the seg fault occurred (which is already in the upper box)

    p i 

this will show what value i has at this point (what is i on your screen and why?)

    quit    

(will quit the gdb debugger and give you back a regular prompt)

Fix the issue based on what you observed. Why did this error occur, and how did gdb make it easier for you to find?

### Troubleshooting Segmentation Faults and Pointer Addresses using GDB

The code in Part 2 is a Binary Search Tree we will study later this semester. We will enter numbers from the command line, which will input values into the tree. For our purposes, we will troubleshoot this code to show the effectiveness of using GDB to trace pointer errors.

To get started, run the following commands:

    > gcc -std=c2x -g -Wall -Wextra -Wconversion -Werror gdb-part2.c -o gdb-part2
    > ./Part2 1 5
    Segmentation fault (core dumped)

How can we use gdb to be able to find the segmentation fault? Well, we can use several commands to be able to track and trace the result. Since we included -g, we can run the commands and then figure out the solution.

    > gdb -tui ./gdb-part2

    list insert

    break insert

    run 1 5

(this starts running the program up to the break point at main, with the inputs 1 and 5, like we would do at the command line)

Once it stops, run the following commands

    next

    p insert_node

(This should print the address in the register pointing to the node)

    p *insert_node

(This should print the addresses of the information in the node)

    next

    p *insert_node

(You will now see the value is updated, as well as the left and right nodes

    next

    p* insert_node

(You will see the C compiler sets the left_node pointer to 0x0 by default, but we did this in class to promote good practices, especially when we get to C++)

    next

    next

    p* insert_node

Now we see that the right_node has a valid address! That should not be. We see in the C code the line:

    insert_node->right = insert_node;


That is not correct. So we get an infinite loop if we try to print. We can see this if you run the command:

    p insert_node

   
You see that the address of insert_node matches insert_node->right. 

    quit

To complete the lab, fix that line of code:

    insert_node->right = NULL;
    
And then compile and run again

    > gcc -std=c2x -g -Wall -Wextra -Wconversion -Werror gdb-part2.c -o gdb-part2
    > ./Part2 1 5  